In [1]:
from google.cloud import bigquery
import pandas as pd

<h3>Notebook feito para o auxilio e aprendizado da utilização do Python com o Google BigQuery</h3>

instalação necessárias:<br>
-pip install pandas<br>
-pip install google-cloud-bigquery<br>

Necessário ter o projeto criado na Google Cloud e a Service Account Key

In [2]:
# Key de acesso:
service_account_json = r"C:\Users\Leopoldo\Desktop\Curso Big Query\big_query_leopoldo.json"

In [3]:
client = bigquery.Client.from_service_account_json(service_account_json)

In [4]:
#Referenciando o projeto
dataset_ref = client.dataset("tabelas_fato", project="curso-big-query-318121")
dataset = client.get_dataset(dataset_ref)

In [5]:
# Print das tabelas do dataset
lista = list(client.list_tables(dataset))
for x in lista:
    print(x.table_id)

dim_cliente
dim_fabrica
dim_organizacional
dim_produto
dim_tempo
fato_presidencia


In [6]:
#Criando a referência da tablea
table_ref = dataset_ref.table("fato_presidencia")

# API request - fetch the table
table = client.get_table(table_ref)

In [7]:
#Verificando as colunas e os tipos de dados
#1º Campo: nome da coluna
#2º Campo: tipo
#3º Campo: Permite nulo ou não
#4º Campo: Descrição do campo
table.schema

[SchemaField('ID_Tempo', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('ID_Fabrica', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('ID_Vendedor', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('ID_Cliente', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('ID_Produto', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('Faturamento', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Unidade_Vendida', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Quantidade_Vendida', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Imposto', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Custo_Variavel', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Custo_Frete', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Custo_Fixo', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Meta_Faturamento', 'FLOAT', 'NULLABLE', None, (), None),
 SchemaField('Meta_Custo', 'FLOAT', 'NULLABLE', None, (), None)]

In [8]:
#Realizando o 'preview' de uma tabela
client.list_rows(table,max_results = 10).to_dataframe()

C:\Users\Leopoldo\anaconda3\lib\site-packages\google\cloud\bigquery\table.py:1860: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,ID_Tempo,ID_Fabrica,ID_Vendedor,ID_Cliente,ID_Produto,Faturamento,Unidade_Vendida,Quantidade_Vendida,Imposto,Custo_Variavel,Custo_Frete,Custo_Fixo,Meta_Faturamento,Meta_Custo
0,1,1,10,4,1,6167.152832,410.0,410.186432,553.751648,820.372864,1921.723389,82.037286,3809.563049,874.134822
1,1,1,10,4,2,3606.521484,342.0,239.875046,323.831329,479.750092,1123.814575,47.975009,2309.626513,505.039315
2,1,1,10,4,3,3390.845703,322.0,225.530136,304.465698,451.060272,1056.608643,45.106027,2138.711460,486.097378
3,1,1,10,4,4,5664.676270,376.0,376.765961,508.634064,753.531921,1765.148560,75.353192,3758.982867,806.654266
4,1,1,10,4,5,5269.815430,350.0,350.503174,473.179291,1402.012695,1642.107422,70.100635,3381.882472,1450.569675
5,1,1,10,4,7,5588.756348,371.0,371.716400,501.817139,1486.865601,1741.491333,74.343280,3636.403932,1543.366991
6,1,1,10,4,8,8242.245117,365.0,548.203918,740.075256,2192.815674,2568.335205,109.640784,5293.184066,2240.108300
7,1,1,10,4,9,3105.748535,295.0,206.567917,278.866669,826.271668,967.770630,41.313583,1975.740840,835.387869
8,1,1,10,4,10,4986.392090,410.0,287.648804,474.620514,2588.839111,1347.634644,57.529761,3483.629001,2555.132343
9,1,1,10,4,12,3651.185059,300.0,210.625031,347.531311,1895.625244,986.778259,42.125006,2506.814118,1849.163650


In [9]:
query = """


SELECT 
dim_tempo.Data, 
dim_fabrica.Desc_Fabrica,
dim_organizacional.Desc_Vendedor as vendedor, dim_organizacional.Desc_Gerente as gerente, dim_organizacional.Desc_Diretor as diretor,
dim_cliente.Desc_Cliente, dim_cliente.Desc_Cidade as cidade_cliente,
dim_produto.Atr_Tamanho, dim_produto.Desc_Categoria, dim_produto.Desc_Marca, dim_produto.Desc_Produto,
ROUND(fato_presidencia.Faturamento,2) as faturamento, ROUND(fato_presidencia.Imposto,2) as imposto, ROUND(fato_presidencia.Custo_Fixo,2) as custo_fixo, 
ROUND(fato_presidencia.Custo_Variavel,2) as custo_variavel, ROUND(fato_presidencia.Meta_Custo,2) as meta_custo, ROUND(fato_presidencia.Meta_Faturamento,2) as meta_faturamento
FROM `curso-big-query-318121.tabelas_fato.fato_presidencia` AS fato_presidencia 
INNER JOIN `curso-big-query-318121.tabelas_fato.dim_fabrica` AS dim_fabrica
ON fato_presidencia.ID_Fabrica = dim_fabrica.ID_Fabrica
INNER JOIN `curso-big-query-318121.tabelas_fato.dim_cliente` AS dim_cliente
ON fato_presidencia.ID_Cliente = dim_cliente.ID_Cliente
INNER JOIN `curso-big-query-318121.tabelas_fato.dim_tempo` as dim_tempo
ON fato_presidencia.ID_Tempo = dim_tempo.ID_Tempo
INNER JOIN `curso-big-query-318121.tabelas_fato.dim_organizacional` as dim_organizacional
ON fato_presidencia.ID_Vendedor = dim_organizacional.ID_Vendedor
INNER JOIN `curso-big-query-318121.tabelas_fato.dim_produto` as dim_produto 
ON fato_presidencia.ID_Produto = dim_produto.ID_Produto
ORDER BY dim_tempo.Data DESC
;
"""


In [10]:
# Criando verificação de tamanho de consultas:
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query_job = client.query(query, job_config = dry_run_config)
consumo_mbytes = round(dry_run_query_job.total_bytes_processed / 1024 / 1024,2)

print(f"Essa consulta consumirá: {consumo_mbytes} MB")

Essa consulta consumirá: 4.79 MB


In [11]:
#Rodando a Query e criando uma tabela
query_job = client.query(query)
resultado = query_job.to_dataframe()

In [12]:
resultado

,Data,Desc_Fabrica,vendedor,gerente,diretor,Desc_Cliente,cidade_cliente,Atr_Tamanho,Desc_Categoria,Desc_Marca,Desc_Produto,faturamento,imposto,custo_fixo,custo_variavel,meta_custo,meta_faturamento
0,2015-11-30,FAbrica do Rio de Janeiro,Willian Show,Taylor Maxwell,Taylor Maxwell,Supermercados Bom e Barato Goiais,Goiais,1 Litro,Aguas Minerais,Linha Citros,Linha Citros - 1 Litro - Lima/Limao,8135.69,754.00,150.80,1508.01,1642.02,4779.54
1,2015-11-30,FAbrica do Rio de Janeiro,Willian Show,Taylor Maxwell,Taylor Maxwell,Supermercados Bom e Barato Goiais,Goiais,700 ml,Aguas Minerais,Linha Citros,Linha Citros - 700 ml - Lima/Limao,5627.42,521.54,104.31,1043.08,1150.24,3803.13
2,2015-11-30,FAbrica do Rio de Janeiro,Willian Show,Taylor Maxwell,Taylor Maxwell,Supermercados Bom e Barato Goiais,Goiais,700 ml,Aguas Minerais,Linha Citros,Linha Citros - 700 ml - Limao,5335.06,494.44,98.89,988.89,1097.79,3475.92
3,2015-11-30,FAbrica do Rio de Janeiro,Willian Show,Taylor Maxwell,Taylor Maxwell,Supermercados Bom e Barato Goiais,Goiais,1 Litro,Aguas Minerais,Linha Citros,Linha Citros - 1 Litro - Limao,8166.01,756.81,151.36,1513.63,1638.77,5139.61
4,2015-11-30,FAbrica do Rio de Janeiro,Willian Show,Taylor Maxwell,Taylor Maxwell,Supermercados Bom e Barato Goiais,Goiais,1 Litro,Aguas Minerais,Linha Refrescante,Linha Refrescante - 1 Litro - Manga,9463.69,900.78,163.78,3821.51,3975.12,5750.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56227,2012-12-31,FAbrica de Sao Paulo,Antonio L,John Campbell,John Campbell,Drogas Super Campo Grande,Campo Grande,700 ml,Suco de Frutas,Festival de Sabores,Festival de Sabores - 700 ml - Maracuja,3580.19,336.23,41.51,830.19,840.22,1934.74
56228,2012-12-31,FAbrica de Sao Paulo,Antonio L,John Campbell,John Campbell,Drogas Super Campo Grande,Campo Grande,350 ml,Suco de Frutas,Light,Light - 350 ml - Melancia,2475.32,269.45,21.39,855.40,856.57,1682.64
56229,2012-12-31,FAbrica de Sao Paulo,Antonio L,John Campbell,John Campbell,Drogas Super Campo Grande,Campo Grande,350 ml,Suco de Frutas,Pedacos de Frutas,Pedacos de Frutas - 350 ml - Maca,2845.08,295.59,27.37,1368.49,1351.52,1806.10
56230,2012-12-31,FAbrica de Sao Paulo,Antonio L,John Campbell,John Campbell,Drogas Super Campo Grande,Campo Grande,700 ml,Suco de Frutas,Videira do Campo,Videira do Campo - 700 ml - Cereja/Maca,5300.79,577.02,45.80,2518.73,2495.36,3401.87


In [13]:
query2 = """
SELECT * FROM `curso-big-query-318121.tabelas_fato.fato_presidencia` AS fato_presidencia 
;
"""

In [14]:
# Suponhamos que eu queira limitar o máximo por consulta:
try:
    maximo_mb = 0.5 * 1024 * 1024  #Resultado em MB
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed =maximo_mb)
    safe_query_job = client.query(query2, job_config=safe_config)
    resultado = safe_query_job.to_dataframe()
except Exception as e :
    print("Consumo acima do estipulado \nErro: \n",e)

Consumo acima do estipulado 
Erro: 
 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/curso-big-query-318121/jobs?prettyPrint=false: Invalid value at 'job.configuration.query.maximum_bytes_billed.value' (TYPE_INT64), "524288.0"

(job ID: 6c6e0e8e-7a9e-476b-9277-9869816ae1ec)

                               -----Query Job SQL Follows-----                                

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT * FROM `curso-big-query-318121.tabelas_fato.fato_presidencia` AS fato_presidencia 
   3:;
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


In [15]:
def funcao_query(query, limite_mb, nome_tabela, nome_projeto):
    """Função conecta no projeto e returna a query especificada se estiver dentro do limite"""
    
    service_account_json = r"C:\Users\Leopoldo\Desktop\Curso Big Query\big_query_leopoldo.json"
    client = bigquery.Client.from_service_account_json(service_account_json)
    
    #Referenciando o projeto
    dataset_ref = client.dataset(nome_tabela, project=nome_projeto)
    dataset = client.get_dataset(dataset_ref)
    
    
    # Criando verificação de tamanho de consultas:
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    dry_run_query_job = client.query(query, job_config = dry_run_config)
    consumo_mbytes = round(dry_run_query_job.total_bytes_processed / 1024 / 1024,2)
    print(f"Essa consulta consumirá: {consumo_mbytes} MB")
    
    if consumo_mbytes < limite_mb:
        query_job = client.query(query)
        resultado = query_job.to_dataframe()
        return resultado
    else:
        print(f"Consulta não realizada, consumo acima do estipulado")
        return False

In [16]:
resultado = funcao_query(query, 0.5, "tabelas_fato", "curso-big-query-318121")

Essa consulta consumirá: 4.79 MB
Consulta não realizada, consumo acima do estipulado


In [17]:
resultado = funcao_query(query2, 10, "tabelas_fato", "curso-big-query-318121")

Essa consulta consumirá: 6.01 MB


In [18]:
resultado

,ID_Tempo,ID_Fabrica,ID_Vendedor,ID_Cliente,ID_Produto,Faturamento,Unidade_Vendida,Quantidade_Vendida,Imposto,Custo_Variavel,Custo_Frete,Custo_Fixo,Meta_Faturamento,Meta_Custo
0,1,1,10,4,1,6167.152832,410.0,410.186432,553.751648,820.372864,1921.723389,82.037286,3809.563049,874.134822
1,1,1,10,4,2,3606.521484,342.0,239.875046,323.831329,479.750092,1123.814575,47.975009,2309.626513,505.039315
2,1,1,10,4,3,3390.845703,322.0,225.530136,304.465698,451.060272,1056.608643,45.106027,2138.711460,486.097378
3,1,1,10,4,4,5664.676270,376.0,376.765961,508.634064,753.531921,1765.148560,75.353192,3758.982867,806.654266
4,1,1,10,4,5,5269.815430,350.0,350.503174,473.179291,1402.012695,1642.107422,70.100635,3381.882472,1450.569675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56227,1065,2,9,57,26,6277.468018,431.0,301.946518,652.204468,2415.572144,2001.905457,120.778607,3724.890075,2569.237234
56228,1065,2,9,57,27,4601.762939,470.0,165.115288,534.973511,1486.037598,1094.714355,66.046115,2881.312134,1565.181697
56229,1065,2,9,57,29,4551.184448,466.0,163.300484,529.093567,1143.103394,1082.682220,65.320193,3212.224213,1218.289519
56230,1065,2,9,57,32,7819.932129,400.0,280.586014,909.098694,2525.274048,1860.285278,112.234405,5184.616028,2661.483659
